In [1]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, models
import kagglehub
import os, shutil

In [2]:
path = kagglehub.dataset_download("davidkingrutgers/cnn-image-classification-dataset")
dest_dir = "/content/dataset"
if not os.path.exists(dest_dir):
    shutil.copytree(path, dest_dir)
print("Dataset copied to:", dest_dir)

Using Colab cache for faster access to the 'cnn-image-classification-dataset' dataset.
Dataset copied to: /content/dataset


In [3]:
base_dir = '/content/dataset/CNN Image Classification Dataset'
train_dir = os.path.join(base_dir,"training")
val_dir = os.path.join(base_dir, "validation")
test_dir = os.path.join(base_dir, "testing")


In [4]:
img_size = (128, 128)
batch_size = 32

train_ds = tf.keras.utils.image_dataset_from_directory(
    train_dir,
    image_size=img_size,
    batch_size=batch_size
)

val_ds = tf.keras.utils.image_dataset_from_directory(
    val_dir,
    image_size=img_size,
    batch_size=batch_size
)
test_ds = tf.keras.utils.image_dataset_from_directory(
    test_dir,
    image_size=img_size,
    batch_size=batch_size
)

Found 1354 files belonging to 3 classes.
Found 1002 files belonging to 3 classes.
Found 1035 files belonging to 3 classes.


In [5]:
print("Classes:", train_ds.class_names)
num_classes = len(train_ds.class_names)
print("Number of classes:", num_classes)

Classes: ['Fruit_479', 'Meat_500', 'Vegetable_374']
Number of classes: 3


In [6]:
normalization_layer = layers.Rescaling(1./255)
train_ds = train_ds.map(lambda x, y: (normalization_layer(x), y))
val_ds = val_ds.map(lambda x, y: (normalization_layer(x), y))
test_ds = test_ds.map(lambda x, y: (normalization_layer(x), y))

train_ds = train_ds.cache().shuffle(1000).prefetch(buffer_size=tf.data.AUTOTUNE)
val_ds = val_ds.cache().prefetch(buffer_size=tf.data.AUTOTUNE)
test_ds = test_ds.cache().prefetch(buffer_size=tf.data.AUTOTUNE)

In [7]:
data_augmentation = tf.keras.Sequential([
    layers.RandomFlip("horizontal"),
    layers.RandomRotation(0.1),
    layers.RandomZoom(0.1),
    layers.RandomContrast(0.1),
])


In [8]:
model = models.Sequential([
    layers.Conv2D(32, (3,3), activation='relu', input_shape=img_size+(3,)),
    data_augmentation,
    layers.MaxPooling2D((2,2)),

    layers.Conv2D(64, (3,3), activation='relu'),
    layers.MaxPooling2D((2,2)),

    layers.Conv2D(128, (3,3), activation='relu'),
    layers.MaxPooling2D((2,2)),

    layers.Flatten(),
    layers.Dense(128, activation='relu'),
    layers.Dropout(0.5),
    layers.Dense(num_classes, activation='softmax')
])

model.summary()

/usr/local/lib/python3.12/dist-packages/keras/src/layers/convolutional/base_conv.py:113: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                 │ (None, 126, 126, 32)   │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ sequential (Sequential)         │ (None, 126, 126, 32)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 63, 63, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 61, 61, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 30, 30, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_2 (Conv2D)               │ (None, 28, 28, 128)    │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_2 (MaxPooling2D)  │ (None, 14, 14, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 25088)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 128)            │     3,211,392 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 3)              │           387 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 3,305,027 (12.61 MB)

 Trainable params: 3,305,027 (12.61 MB)

 Non-trainable params: 0 (0.00 B)

In [10]:
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

history = model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=30
)


Epoch 1/30
42/43 ━━━━━━━━━━━━━━━━━━━━ 0s 44ms/step - accuracy: 0.4175 - loss: 1.1325

/usr/local/lib/python3.12/dist-packages/keras/src/trainers/epoch_iterator.py:160: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self._interrupted_warning()


43/43 ━━━━━━━━━━━━━━━━━━━━ 13s 169ms/step - accuracy: 0.4185 - loss: 1.1305 - val_accuracy: 0.6357 - val_loss: 0.8277
Epoch 2/30
43/43 ━━━━━━━━━━━━━━━━━━━━ 2s 48ms/step - accuracy: 0.6141 - loss: 0.8384 - val_accuracy: 0.6467 - val_loss: 0.7939
Epoch 3/30
43/43 ━━━━━━━━━━━━━━━━━━━━ 2s 46ms/step - accuracy: 0.7034 - loss: 0.6667 - val_accuracy: 0.6607 - val_loss: 0.7228
Epoch 4/30
43/43 ━━━━━━━━━━━━━━━━━━━━ 2s 44ms/step - accuracy: 0.7277 - loss: 0.6401 - val_accuracy: 0.6397 - val_loss: 0.6874
Epoch 5/30
43/43 ━━━━━━━━━━━━━━━━━━━━ 2s 44ms/step - accuracy: 0.7072 - loss: 0.6349 - val_accuracy: 0.7226 - val_loss: 0.6352
Epoch 6/30
43/43 ━━━━━━━━━━━━━━━━━━━━ 2s 44ms/step - accuracy: 0.7828 - loss: 0.5369 - val_accuracy: 0.7285 - val_loss: 0.6250
Epoch 7/30
43/43 ━━━━━━━━━━━━━━━━━━━━ 2s 44ms/step - accuracy: 0.7692 - loss: 0.5375 - val_accuracy: 0.7405 - val_loss: 0.6249
Epoch 8/30
43/43 ━━━━━━━━━━━━━━━━━━━━ 2s 47ms/step - accuracy: 0.7830 - loss: 0.5306 - val_accuracy: 0.7645 - val_loss: 

In [11]:
loss, acc = model.evaluate(val_ds)
print(f"Validation Accuracy: {acc:.2f}")


import numpy as np
for images, labels in val_ds.take(1):
    preds = model.predict(images)
    print("Predicted:", np.argmax(preds[0]), " | Actual:", labels[0].numpy())

32/32 ━━━━━━━━━━━━━━━━━━━━ 8s 270ms/step - accuracy: 0.7929 - loss: 0.6802
Validation Accuracy: 0.79
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 97ms/step
Predicted: 1  | Actual: 1


In [12]:
model.save("fruit_meat_vegetable_cnn_model.h5")